# Projet : *California Housing Prices* (Prix des logements en Californie)

In [1]:
# pour utiliser des formules mathématiques en markdown
#!pip install -U sympy

## Contexte & Data

Nous utilisons le jeu de données California Housing Prices de l'entrepôt StatLib. Ces données proviennent du recensement californien de 1990 (bien sûr, elles ne sont pas actuelles).

## Prendre du recul sur l'ensemble

La mission consiste à construire un modèle (Modèle d'apprentissage automatique; *Machine learning* ou ML) des prix immobiliers en Californie en se basant sur les données du recensement (par exemple : population, revenu, prix médian des habitations pour chaque bloc de 600 à 3000 personnes, également appelé "distinct").

Le modèle doit être capable de prédire le prix médian des habitations dans chaque **distinct** en fonction des autres variables.

### 1. Cerner le problème

Les questions à se poser :
- Quel est l'objectif fonctionnel ?
- Comment l'entreprise compte-t-elle utiliser ce modèle et en tirer parti ?
- Existe-t-il une solution à l'heure actuelle ?

Dans notre cas, la sortie du modèle (prédiction du prix médian des habitations dans un distinct) servira à alimenter un autre système ML. Ce système en aval utilisera les résultats de notre modèle (avec d'autres signaux) pour déterminer s'il est intéressant ou non d'investir dans un distinct donné (Analyse financière avant investissement). La prédiction va impacter le chiffre d'affaires (CA) de l'entreprise. Une équipe rassemble les informations récentes sur les distincts, estime si nécessaire le prix médian selon des règles de calcul complexes (avec un écart de 20%). D'où l'intérêt d'utiliser un modèle de prédiction.

Il s'agit d'une tâche classique **d'apprentissage supervisé** (données **étiquetées** : pour chaque distinct, on a les données avec le résultat attendu : prix médian). On va prédire une valeur à l'aide de plusieurs variables : il s'agit donc d'une **régression multiple**. C'est un problème de **régression univariée** car on prédit une seule valeur pour chaque distinct. On a un jeu de données qui tient en mémoire, et pas besoin d'avoir des résultats en continu, on fera donc un **apprentissage groupé (batch learning)** pour aborder le sujet.

### 2. Comment mesurer la performance de mon modèle ?

Nous utiliserons la racine carrée de l'erreur quadratique moyenne (RMSE) pour évaluer la performance de notre modèle de régression. La formule du RMSE (fonction de coût) est la suivante :

$$
RMSE(X, h) = \sqrt{\frac{1}{m} \sum_{i=1}^{m} (h(x^{(i)}) - y^{(i)})^2}
$$

ou :
- $ X $ est la matrice contenant toutes les valeurs des variables (à l'exception des étiquetes)
- $(m)$ est le nombre d'observations dans le dataset.
- $(x^{(i)})$ est le vecteur des caracteristiques de la i-eme instance dans le dataset.
- $(y^{(i)}$) est l'etiquette (la valeur de sortie souhaitée) de la i-eme instance dans le dataset.
- $(h)$ est la fonction de prediction (le modele), qui donne la prediction pour $(x^{(i)})$ (notee $(h(x^{(i)}))$).

Cette formule calcule la difference entre la prediction du modele $(h(x^{(i)}))$ et la valeur reelle $(y^{(i)})$ pour chaque instance dans le dataset, eleve cette difference au carré, puis prend la moyenne de toutes ces valeurs. Enfin, elle prend la racine carrée de cette moyenne pour obtenir le $RMSE$.

## Charger les données

In [27]:
import os
import tarfile
import urllib
import json
import pandas as pd
import logging

# Configuration du logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Charger le fichier de configuration
with open('conf/config.json') as f:
    config = json.load(f)

# Accéder aux variables
DOWNLOAD_ROOT = config['DOWNLOAD_ROOT']
HOUSING_PATH = os.path.join("datasets", "housing")

HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    """
    Crée le dossier datasets/housing si nécessaire, télécharge le zip housing.tgz et extrait le housing.csv
    """
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    logger.info("Téléchargement du fichier housing.tgz depuis %s", housing_url)
    urllib.request.urlretrieve(housing_url, tgz_path)
    logger.info("Extraction du fichier housing.tgz")
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    logger.info("Le fichier housing.tgz a été extrait avec succès")

def load_housing_data(housing_path=HOUSING_PATH):
    """
    Lit le fichier csv
    """
    csv_path = os.path.join(housing_path, "housing.csv")
    logger.info("Chargement du fichier housing.csv depuis %s", csv_path)
    return pd.read_csv(csv_path)


In [29]:
# Appel des fonctions
fetch_housing_data()
housing_data = load_housing_data()

INFO:__main__:Téléchargement du fichier housing.tgz depuis https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.tgz
INFO:__main__:Extraction du fichier housing.tgz
INFO:__main__:Le fichier housing.tgz a été extrait avec succès
INFO:__main__:Chargement du fichier housing.csv depuis datasets\housing\housing.csv


In [30]:
housing_data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [31]:
housing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB
